In [1]:
%pylab inline
import json
from textgen import *
m = TextGen(
    img_size=299, 
    code_dim=512, 
    rnn_type='LSTM', 
    cnn_type='InceptionV3',
    pooling='avg',
)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def validate(model, val_record, ckpt, batch_size=32, capacity=8, n_threads=6, n_samples=10000, 
             inds_to_words_json='/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/inds_to_words.json'):
    ''''''
    with tf.variable_scope('BatchReader'):
        print('creating batch reader')
        coord =  tf.train.Coordinator()

        # read and preprocess training records
        x, y, c, id_ = model.read_tfrecord(
            val_record, 
            batch_size=batch_size, 
            capacity=capacity, 
            n_threads=n_threads,
            n_epochs=None, 
            shuffle=False)
        x = model.preproc_img(x)
        y, _ = model.preproc_caption(y, random=False)

    with tf.variable_scope('Strings'):
        print('getting strings')

        y_hat = model.sample_recursively(x, batch_size=batch_size, 
                                         continuous=False)
        table = model.create_table(inds_to_words_json)
        y_strings = model.postproc_caption(y, table)
        y_hat_strings = model.postproc_caption(y_hat, table)

    with tf.Session(graph=model.graph) as sess:

        print('initializing variables')
        sess.run(tf.global_variables_initializer())

        print('loading weights from '+ckpt)
        model.load_weights(ckpt)
        
        print('starting threads')
        tf.train.start_queue_runners(sess=sess, coord=coord)

        n = 0
        results = []
        try:
            for _ in tqdm.trange(n_samples // batch_size):
                if not coord.should_stop():
                    s_hat, s, ids = sess.run([y_hat_strings, y_strings, id_], {backend.learning_phase(): False})
                    s_hat, s = [[c.replace('SOS ', '').replace(' EOS', '') \
                                 for c in b] for b in s_hat, s]
                    for i in range(batch_size):
                        d = {"image_id": ids[i], "caption": s_hat[i]}
                        results.append(d)
                    n += batch_size
                else:
                    raise IOError, 'queues closed'
            coord.request_stop()
            time.sleep(0.2)
            return results
        # exit behaviour: request thread stop, then wait for 
        # them to recieve message before exiting session context
        except:
            coord.request_stop()
            time.sleep(0.2)
            raise

In [3]:
results = validate(m, 
 '/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/val_1shot_NOVEL.tfrecord',
 '/home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000550000.h5', 
n_samples=10000,
batch_size=64)
with open('novel.json', 'w') as fout:
    json.dump(results, fout)

creating batch reader
getting strings
initializing variables
loading weights from /home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000550000.h5


  0%|          | 0/156 [00:00<?, ?it/s]

starting threads


100%|██████████| 156/156 [01:54<00:00,  1.52it/s]


In [4]:
results = validate(m, 
 '/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/val_1shot_BASE.tfrecord',
 '/home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000550000.h5', 
n_samples=10000,
batch_size=64)
with open('base.json', 'w') as fout:
    json.dump(results, fout)

creating batch reader
getting strings
initializing variables
loading weights from /home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000550000.h5


  0%|          | 0/156 [00:00<?, ?it/s]

starting threads


100%|██████████| 156/156 [02:03<00:00,  1.50it/s]


In [5]:
results = validate(m, 
 '/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/val_1shot.tfrecord',
 '/home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000550000.h5', 
n_samples=10000,
batch_size=64)
with open('all.json', 'w') as fout:
    json.dump(results, fout)

creating batch reader
getting strings
initializing variables
loading weights from /home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000550000.h5


  0%|          | 0/156 [00:00<?, ?it/s]

starting threads


100%|██████████| 156/156 [02:17<00:00,  1.50it/s]
